# Flower Classifier

A Tensor Flow classification model to classify the type of flower ('Setosa', 'Versicolor', or 'Virginica') given its sepal and petal properties ('SepalLength', 'SepalWidth', 'PetalLength', and 'PetalWidth')

In [11]:
from __future__ import absolute_import, division, print_function, unicode_literals
# Just need TensorFlow and Pandas
import tensorflow as tf
import pandas as pd

## Load data

Retrieve the data from the Kaggle source using TensorFlow's Keras module to get t

In [12]:
# Set the features and the labels (the species of the flower)
FEATURES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
LABELS = ['Setosa', 'Versicolor', 'Virginica']

# Get the files from the URLs
train_file = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_file = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

# Read the CSVs into memory
train_set = pd.read_csv(train_file, names=FEATURES, header=0)
test_set = pd.read_csv(test_file, names=FEATURES, header=0)

# Get the labels for both the data sets
train_labels = train_set.pop('Species')
test_labels = test_set.pop('Species')


# Display the data
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


## Input function

A simple input function to the classifier model to divide the dataset into batches with a default size of 256

In [13]:
def model_input_fn(features, labels, training=True, batch_size=256):
    # Convert to a dataset format 
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # If still training, shuffle the dataset and repeat
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

feature_columns = []
for key in train_set.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
    

## The Model

We'll be using the powerful Deep-Neural-Network Classifier as it suits our purpose perfectly

In [15]:
# Create the model
flower_classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    # 30 x 10
    hidden_units=[30, 10],
    # Three species so three classes
    n_classes=3)

# Train the model
flower_classifier.train(
    input_fn=lambda: model_input_fn(train_set, train_labels, training=True),
    steps=6000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tejas\\AppData\\Local\\Temp\\tmpgd9a8w1q', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
I

Check the accuracy of the model on the test_set

In [28]:
test_result = flower_classifier.evaluate(
    input_fn=lambda: model_input_fn(test_set, test_labels, training=False))

print('The accuracy for the model is: {accuracy:0.4f}\n'.format(**test_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-06-21T22:21:15Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\tejas\AppData\Local\Temp\tmpgd9a8w1q\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.26729s
INFO:tensorflow:Finished evaluation at 2020-06-21-22:21:16
INFO:tensorflow:Saving dict for global step 6000: 

There should be an accuracy around 96%, which is not bad. Now, we can try predicting on some test cases.

In [30]:
def model_input_without_labels(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

test_values = {
    'SepalLength': [2.1, 7.6, 3.3],
    'SepalWidth': [7.5, 8.9, 3.5],
    'PetalLength': [2.75, 6.82, 7.4],
    'PetalWidth': [3.5, 0.85, 5.5],
}

predictions = flower_classifier.predict(input_fn=lambda: model_input_without_labels(test_values))
for pred in predictions:
    class_id = pred['class_ids'][0]
    probability = pred['probabilities'][class_id]

    print('This flower is a  "{}" with a chance of ({:.1f}%)'.format(LABELS[class_id], 100 * probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\tejas\AppData\Local\Temp\tmpgd9a8w1q\model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
This flower is a  "Setosa" with a chance of (51.0%)
This flower is a  "Versicolor" with a chance of (67.0%)
This flower is a  "Virginica" with a chance of (99.5%)
